# Model tuning

Após selecionarmos as melhores features para nosso modelo, o que faremos nesse notebook será o tuning de hiperparametros do lightgbm. A ideia é encontrar o melhor conjunto de parametros para trazer algum ganho incremental de performance na nossa solução e, principalmente, encontrar parâmetros que diminuam o risco de overfitting do modelo.

Essa redução no risco de overfitting de um modelo de árvore se dá devido a alguns fatores, como:
* Controle no processo de crescimento das árvores;
* Controle na quantidade de nós de decisão das árvores;
* Restrição na quantidade de samples e features que, aleatoriamente, o algoritmo irá usar para treinar as árvores em cada iteração;
* Regularização da função objetivo através dos parametros reg_alpha e reg_lambda, por exemplo.

Para reduzir o tempo de execução, iremos definir um vasto espaço de busca e utilizar o RandomizedSearchCV do scikit-learn que, aleatoriamente, irá testar algumas combinações. Em alguns casos, quando se tem uma melhor ideia de como construir o espaço de busca, também pode ser utilizado o GridSearchCV, que testa todas as combinações possíveis dentro de um grid. Outra alternativa também é usar algoritmos de otimização como o hyperopt e optuna para decidir os melhores parâmetros

In [1]:
#importando as bibliotecas que serão utilizadas no processo
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_val_score

import FeatureGenerator as fg

from lightgbm import LGBMClassifier

In [2]:
#carregando os dados de treino
data_path = "../inputs/train.csv"
df_train = pd.read_csv(data_path)
df_train.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,target
0,28104,50000.0,2,1,1,31,1,2,2,0,...,50332.0,29690.0,30246.0,2200.0,4.0,2300.0,1100.0,1400.0,1200.0,1
1,29094,330000.0,2,2,2,59,0,0,0,0,...,80589.0,76180.0,61693.0,20000.0,3500.0,19000.0,15000.0,3000.0,2139.0,0
2,11280,220000.0,2,1,2,41,-1,-1,-2,-2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,28435,50000.0,2,2,1,45,0,0,0,0,...,8469.0,8411.0,8361.0,2124.0,2037.0,1130.0,295.0,302.0,296.0,0
4,10873,480000.0,2,3,1,42,-2,-2,-2,-2,...,0.0,790.0,0.0,0.0,0.0,0.0,790.0,0.0,0.0,0


In [3]:
#selecionando as variáveis categóricas para definir no modelo
cat_cols = ['SEX',
 'EDUCATION',
 'MARRIAGE',
 'PAY_0',
 'PAY_2',
 'PAY_3',
 'PAY_4',
 'PAY_5',
 'PAY_6']

In [12]:
#criando uma lista com as features que foram selecionadas 
selected_features = ['BILL_AMT2_minus_BILL_AMT1',
 'PAY_AMT5/BILL_AMT5',
 'PAY_AMT6/BILL_AMT6',
 'PAY_AMT1/BILL_AMT1',
 'BILL_AMT5_minus_BILL_AMT4',
 'PAY_AMT4/BILL_AMT4',
 'PAY_AMT3/BILL_AMT3',
 'AGE',
 'BILL_AMT3_minus_BILL_AMT2',
 'BILL_AMT4_minus_BILL_AMT3',
 'PAY_AMT6',
 'PAY_AMT6_minus_PAY_AMT5',
 'PAY_AMT2/BILL_AMT2',
 'BILL_AMT6_minus_BILL_AMT5',
 'PAY_0',
 'LIMIT_BAL/EDUCATION_mean',
 'PAY_AMT2',
 'LIMIT_BAL',
 'PAY_AMT1',
 'PAY_AMT2_minus_PAY_AMT1',
 'PAY_AMT4_minus_PAY_AMT3',
 'PAY_AMT3_minus_PAY_AMT2',
 'PAY_AMT3',
 'BILL_AMT1/EDUCATION_max',
 'PAY_AMT4']

In [13]:
#inicializando as classes que serão utilizadas para geração de features
#a ideia de utilizar classes é poder encapsular tudo num só pipeline

#classe para criar as features de divisão
fbf = fg.FeatureByFeature(features_num=["PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6"],
                         features_denom=["BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6"])

#classe para criar as features de diferença
diff1 = fg.DiffFeatures(features=["PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6"])

diff2 = fg.DiffFeatures(features=["BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6"])

#classe para criar as features de agrupamento
agpf = fg.GroupFeatures(group_columns=["EDUCATION"], features=["LIMIT_BAL", "BILL_AMT1", 
                                                               "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", 
                                                               "BILL_AMT5", "BILL_AMT6"])
#classe para filtrar as features que serão usadas como input
final_features = fg.FinalFeatures(features=selected_features)

In [14]:
#criando input e output
X_train, y_train = df_train.drop(columns=["ID", "target"], axis=1), df_train.target.values

In [15]:
#inicializando o lightgbm
base_estimator = LGBMClassifier(class_weight="balanced",
                               random_state=42)

In [16]:
#definindo o nosso pipeline com as transformações e o algoritmo final
pipe = Pipeline(steps=[("FeatureByFeature", fbf),
                      ("diff_features1", diff1),
                      ("diff_features2", diff2),
                      ("aggrouped_features", agpf),
                      ("FinalFeatures", final_features),
                      ("Estimator", base_estimator)])

In [9]:
#vamos utilizar a validação cruzada para avaliar o modelo em diferentes partições do dataset
cross_val = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [17]:
#resultados da validação cruzada
print(cross_val_score(pipe, X_train, y_train, cv=cross_val, scoring="f1"))

[0.54536489 0.51815017 0.53200569 0.50986193 0.54518664]


In [18]:
#resultado médio da validação cruzada
print(np.mean([0.54536489, 0.51815017, 0.53200569, 0.50986193, 0.54518664]))

0.5301138639999999


In [ ]:
#aqui definimos o nosso grid de busca onde o tuning de parametros buscará as melhores combinações
distributions = {"Estimator__n_estimators": [100, 200, 300, 500, 650, 800, 1000, 1200, 1600, 2000],
                "Estimator__colsample_bytree": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, None],
                "Estimator__max_depth": [2, 4, 6, 8, 10, 12, 16, None],
                "Estimator__num_leaves": [8, 10, 12, 16, 20, 31, 41, 51, 61],
                "Estimator__min_child_samples": [20, 30, 40, 50, 60, 80, 100],
                "Estimator__reg_alpha": [2, 4, 6, 8, 10, 12, 16],
                "Estimator__reg_lambda": [2, 4, 6, 8, 10, 12, 16],
                "Estimator__learning_rate": [0.3, 0.1, 0.08, 0.06, 0.04, 0.03, 0.02, 0.01, 0.008, 0.005]}

In [ ]:
#inicializando o random search
clf = RandomizedSearchCV(pipe, distributions, cv=cross_val, scoring="f1")

In [ ]:
#fit
clf.fit(X_train, y_train)

In [ ]:
#Vamos analisar os 3 melhores resultados e ver  os parâmetros que iremos utilizar adiante
results = pd.DataFrame(clf.cv_results_)
results = results.sort_values(by="rank_test_score", ascending=True)
results.head(3)

In [ ]:
#melhores parametros do processo de tuning
clf.best_params_

In [ ]:
#criando nosso dicionário de parâmetros para configurar o modelo
selected_params = {'reg_lambda': 10,
 'reg_alpha': 2,
 'num_leaves': 12,
 'n_estimators': 800,
 'min_child_samples': 40,
 'max_depth': 2,
 'learning_rate': 0.02,
 'colsample_bytree': None}

In [ ]:
#inicializando um novo lightgbm, agora com os parâmetros escolhidos
estimator = LGBMClassifier(**selected_params,
                           class_weight="balanced",
                           random_state=42)

In [ ]:
#configurando o pipeline com o novo modelo inicializado
pipe = Pipeline(steps=[("FeatureByFeature", fbf),
                      ("diff_features1", diff1),
                      ("diff_features2", diff2),
                      ("aggrouped_features", agpf),
                      ("FinalFeatures", final_features),
                      ("Estimator", estimator)])

In [ ]:
#usando a validação cruzada para avaliar o novo modelo
print(cross_val_score(pipe, X_train, y_train, cv=cross_val, scoring="f1"))

In [ ]:
#resultado médio da validação cruzada
print(np.mean([0.54968492, 0.50482625, 0.53438662, 0.53012048, 0.55875183]))

Parece que obtivemos um resultado melhor com o novo modelo em comparação com o modelo default, então vamos seguir utilizando-o adiante.

### Seleção de threshold

Outro parâmetro importante em um modelo de classificação é o threshold de probabilidade que o modelo está utilizando para decidir se um determinado input deve ser classificado como 0 ou 1. Aqui vamos avaliar em que valor esse corte deve estar para gerar os melhores resultados

In [ ]:
#função para converter probabilidade para classe
def prob2class(value, thresh):
    if value > thresh:
        return 1
    else:
        return 0

In [ ]:
#vamos carregar uma base de validação para fazer o ajuste de threshold
eval_set = pd.read_csv("../inputs/validation.csv")

In [ ]:
#criando input e output de validação
X_val, y_val = eval_set.drop(columns="target", axis=1), eval_set.target.values

In [ ]:
#fit do pipeline nos dados de treino
pipe.fit(X_train, y_train)

In [ ]:
#gerando as predições de probabilidade
#vamos criar uma coluna que contém as probabilidades como informação
y_proba = pipe.predict_proba(X_val)
y_proba = [p[1] for p in y_proba]
X_val["probability"] = y_proba

In [ ]:
#definindo o espaço de análise dos thresholds
thresholds = [0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85]
f1s = []

In [ ]:
#laço para calcular a performance do modelo em diferentes thresholds
for t in thresholds:
    X_val["prediction"] = X_val.apply(lambda x: prob2class(x["probability"], thresh=t), axis=1)
    y_pred = X_val["prediction"].values
    
    f1 = f1_score(y_val, y_pred)
    f1s.append(f1)

In [ ]:
#plotando o gráfico de threshold vs f1
plt.figure(figsize=(10, 4))
sns.lineplot(x=thresholds, y=f1s, marker='o')
plt.xlabel("thresholds")
plt.ylabel("f1_score")
plt.title("Threshold selection for f1_score")

O nosso threshold final então está em 0.55. Note que a definição do threshold foi realizada com base em um dataset de validação. Nesse caso, o mais seria ter uma terceira partiçao de dados para avaliar o quanto que, nessas condições que foram encontradas, o modelo é capaz de generalizar em dados nunca vistos.

# Outras abordagens

Para tentar incrementar a performance do nosso modelo, poderíamos testar outras técnicas como as arquiteturas de ensemble, como voting e stacking utilizando outros modelos como catboost e xgboost em combinação com o lightgbm.